In [2]:
print('test')

test


In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm.autonotebook import tqdm
from joblib import Parallel, delayed
import umap
import pandas as pd

/mnt/cube/tsainbur/conda_envs/tpy3/lib/python3.6/site-packages/tqdm/autonotebook/__init__.py:14: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [5]:
from avgn.utils.paths import DATA_DIR, most_recent_subdirectory, ensure_dir

In [6]:
DATASET_ID = 'mobysound_humpback_whale'

In [7]:
from avgn.utils.hparams import HParams
from avgn.dataset import DataSet

In [8]:
from avgn.signalprocessing.create_spectrogram_dataset import prepare_wav, create_label_df, get_row_audio

### create dataset

In [9]:
hparams = HParams(
    num_mel_bins = 32,
    mel_lower_edge_hertz=1,
    mel_upper_edge_hertz=1995,
    butter_lowcut = 1,
    butter_highcut = 1995,
    ref_level_db = 20,
    min_level_db = -25,
    win_length_ms = 100,
    hop_length_ms = 20,
    mask_spec = True,
    mask_spec_kwargs = {"spec_thresh": 0.25, "offset": 1e-10},
    reduce_noise=True,
    noise_reduce_kwargs = {"n_std_thresh": 1.5, "prop_decrease":0.9}

)

In [10]:
# create a dataset object
dataset = DataSet(DATASET_ID, hparams = hparams)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 24 concurrent workers.


[Parallel(n_jobs=-1)]: Done   8 out of  13 | elapsed:    7.1s remaining:    4.5s
[Parallel(n_jobs=-1)]: Done  13 out of  13 | elapsed:    7.2s finished


In [11]:
dataset.sample_json

OrderedDict([('bout_number', 7),
             ('datetime', '1994-03-05_10-37-00'),
             ('samplerate_hz', 4000),
             ('length_s', 255.65),
             ('wav_loc',
              '/mnt/cube/Datasets/mobysound/humpback-01/940305-1037.wav'),
             ('noise_loc',
              '/mnt/cube/tsainbur/Projects/github_repos/avgn_paper/data/processed/mobysound_humpback_whale/2019-06-23_20-56-52/NOISE/940305-1037__h0m4s0.649993896484375.WAV'),
             ('indvs',
              OrderedDict([('UNK',
                            OrderedDict([('syllables',
                                          OrderedDict([('start_times',
                                                        [0.6800000071525574,
                                                         2.5899999141693115,
                                                         6.400000095367432,
                                                         8.0600004196167,
                                                     

In [25]:
# for each wav, compute the spectrogram, break the spectrogram up into multiple segments, then project into PCA, then plot PCA projections over time